In [60]:
import pandas as pd
import os
from json import loads, dumps
import tempfile, shutil

os.chdir(os.getcwd())
file_path_1 = 'FinalEmbeddingResults/videosummaries_file_13082024.json'
file_path_2 ='FinalEmbeddingResults/summarycontent_image_embeddings_v1.json'

df_video = pd.read_json(file_path_1, lines=True)
df_video=df_video.rename(columns={"video path": "path", 
                        "chapter_summary": "content",
                        "chapter_embedding": "content_embedding",
                       })
df_video['media_type']=['video']*len(df_video)

df_image = pd.read_json(file_path_2, lines=True)[['id',	"image path",	'summary',	'summary embedding']]
df_image=df_image.rename(columns={"image path": "path",
                                  "summary":'content',
                                  "summary embedding": "content_embedding",
                       })
df_image['media_type']=['image']*len(df_image)
df_content = pd.concat([df_video,df_image],axis=0 )

df_content.head()

,id,path,content,content_embedding,start_offset_sec_chapter,end_offset_sec_chapter,media_type
0,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,A woman in a sparkling dress walks down a corr...,"[0.06952495872974396, -0.02421196736395359, -0...",0.0,16.0,video
1,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,Madeleine West is an Australian actress. She r...,"[-0.007446157746016979, -0.025421058759093285,...",16.0,32.0,video
2,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,"Madeleine West talks about her abuser, Peter W...","[-0.011637571267783642, -0.028489504009485245,...",32.0,48.0,video
3,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,Detective Scott Tudnem is a Victorian police o...,"[-0.04620090499520302, -0.040192730724811554, ...",48.0,64.0,video
4,60MI24_1_A_HBB.mp4,gs://raw_nine_files/60MI24_1_A_HBB.mp4,Madeleine West says the abuse started when she...,"[0.010253753513097763, -0.019587762653827667, ...",64.0,80.0,video


In [51]:
result = df_content.to_json(orient="records")
parsed = loads(result)
json_content=dumps(parsed, indent=4)  


# Create temporary file to write summaries to
content_embeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)
with  open(content_embeddings_file.name, "a") as sf:     
  sf.writelines(json_content) 

content_embeddings_file_name = content_embeddings_file.name
content_embeddings_file.close()
shutil.copy(content_embeddings_file_name, 'content_embeddings.json')


'content_embeddings.json'

### create bucket and load data into bucket

In [52]:
#set project info
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

In [55]:
#create bucket if not exists

from google.cloud import storage
from google.cloud.storage import Bucket
bucket_name=f"artifacts-{PROJECT_ID}-embeddings"
client = storage.Client()
exists = Bucket(client, bucket_name).exists()

if not exists:
    print('creating bucket..')
    #set bucket info to create a bucket
    BUCKET_URI = f"gs://{bucket_name}"  # @param {type:"string"}
    ! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://artifacts-nine-quality-test-embeddings/...


In [56]:
# #embeddings
# UNIQUE_FOLDER_NAME='content_embeddings'
# EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
# ! gsutil cp {content_embeddings_file.name} {EMBEDDINGS_INITIAL_URI}


Copying file:///var/tmp/tmp1c1lbcdr.json [Content-Type=application/json]...
/ [1 files][  5.3 MiB/  5.3 MiB]                                                
Operation completed over 1 objects/5.3 MiB.                                      


In [57]:
# #embeddings
# UNIQUE_FOLDER_NAME='content_embeddings'
# EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
# ! gsutil cp content_embeddings.json {EMBEDDINGS_INITIAL_URI}


Copying file://content_embeddings.json [Content-Type=application/json]...
/ [1 files][  5.3 MiB/  5.3 MiB]                                                
Operation completed over 1 objects/5.3 MiB.                                      


In [58]:
#we have to remove [] from json file to be able to load it into bigquery

!cat content_embeddings.json | jq -c '.[]' > content_embeddings_V1.json

In [59]:
#embeddings
UNIQUE_FOLDER_NAME='content_embeddings'
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp content_embeddings_V1.json {EMBEDDINGS_INITIAL_URI}

Copying file://content_embeddings_V1.json [Content-Type=application/json]...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      
